In [3]:
import requests
from bs4 import BeautifulSoup

# Beautiful Soup Scraper

## Yahoo Finance

In [ ]:
import requests
from bs4 import BeautifulSoup
import datetime
from datetime import datetime, date, timedelta
import re
import pandas as pd

def convert_date(durationPosted):
    """Function to convert `jobs['datePosted']` into the actual date it was posted"""

    if durationPosted is None:
        datePosted = None
        return datePosted
    else: 
        today = date.today()

        # Testing for 'hours'
        hours_pattern = r"(\d+)\s*hours?\s*ago"
        hourMatched = re.search(hours_pattern, durationPosted)

        # Testing for 'day'
        day_pattern = r"(\d+)\s*days?\s*ago"
        dayMatched = re.search(day_pattern, durationPosted)

        # Testing for 'week'
        week_pattern = r"(\d+)\s*weeks?\s*ago"
        weekMatched = re.search(week_pattern, durationPosted)

        # Handling 'today'
        if "today" in durationPosted.lower():
            datePosted = today
        elif hourMatched:
            datePosted = today - timedelta(hours=int(hourMatched.group(1)))
        elif dayMatched:
            datePosted = today - timedelta(days=int(dayMatched.group(1)))
        elif weekMatched:
            datePosted = today - timedelta(weeks=int(weekMatched.group(1)))
        else:
            datePosted = None

        # Check if datePosted is None before calling strftime
        if datePosted is not None:
            return datePosted.strftime("%m/%d/%Y")
        else:
            return None


def get_news_data(ticker):
    url = f"https://finance.yahoo.com/quote/{ticker}/news"
    response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
    soup = BeautifulSoup(response.text, "html.parser")

    # For text processing
    delimiter = "•"

    # Intialize an empty dataframe
    scraped_df = pd.DataFrame(columns=["Title","Summary","Date","Link"])

    # Extract news articles
    articles = soup.find_all("li", {"class": "story-item"})
    for article in articles:
        title = article.find("h3").text
        link = article.find("a")["href"]
        summary = article.find("p").text if article.find("p") else ""

        # Getting the date 
        published = article.find("div", {"class": ["publishing","yf-1weyqlp"]}).text
        # date = date[sep_index:]
        published = published.split(delimiter)[1].strip()
        published = convert_date(published)

        # Conduct error handling; if EXACT days was not found...
        if not published: # published = False
            article_response = requests.get(link, headers={"User-Agent": "Mozilla/5.0"})
            article_soup = BeautifulSoup(article_response.text, "html.parser")
            published = article_soup.find("time", "byline-attr-meta-time")
            # print(published) # DEBUG
            published = datetime.strptime(published.text, '%a, %b %d, %Y, %I:%M %p').date()

        # Place it into a dataframe
        row_article = pd.DataFrame([{"Title" : title, "Summary": summary, "Date": published, "Link" : link}])
        scraped_df = pd.concat([scraped_df, row_article], ignore_index=True)
        
    return scraped_df

# Ref: 
# https://scrapfly.io/blog/guide-to-yahoo-finance-api/

In [ ]:
eur_usd_df = get_news_data("EURUSD=X")
today = date.today()
eur_usd_df.to_csv(f"EURUSD=X_{today}")

<time class="byline-attr-meta-time" data-timestamp="2025-02-06T15:07:59.000Z" datetime="2025-02-06T15:07:59.000Z">Thu, Feb 6, 2025, 8:07 AM</time>
<time class="byline-attr-meta-time" data-timestamp="2025-02-03T10:33:47.000Z" datetime="2025-02-03T10:33:47.000Z">Mon, Feb 3, 2025, 3:33 AM</time>
<time class="byline-attr-meta-time" data-timestamp="2025-01-31T10:44:26.000Z" datetime="2025-01-31T10:44:26.000Z">Fri, Jan 31, 2025, 3:44 AM</time>
<time class="byline-attr-meta-time" data-timestamp="2025-01-24T10:52:23.000Z" datetime="2025-01-24T10:52:23.000Z">Fri, Jan 24, 2025, 3:52 AM</time>
<time class="byline-attr-meta-time" data-timestamp="2025-01-22T10:24:12.000Z" datetime="2025-01-22T10:24:12.000Z">Wed, Jan 22, 2025, 3:24 AM</time>
<time class="byline-attr-meta-time" data-timestamp="2025-01-17T09:47:21.000Z" datetime="2025-01-17T09:47:21.000Z">Fri, Jan 17, 2025, 2:47 AM</time>
<time class="byline-attr-meta-time" data-timestamp="2024-12-13T16:04:22.000Z" datetime="2024-12-13T16:04:22.000Z">

In [3]:
import pandas as pd
pd.read_csv("JPY=X_2025-03-10.csv")

,Unnamed: 0,Title,Summary,Date,Link
0,0,EU Sovereigns Face Multiple Risks to Credit Ou...,"Rising tariffs, lower growth, higher defence s...",02/18/2025,https://finance.yahoo.com/news/eu-sovereigns-f...
1,1,The Dollar Fluctuates Close to Recent Highs Ah...,Tariffs and politics remain in focus.,2025-02-06,https://finance.yahoo.com/news/dollar-fluctuat...
2,2,Will the U.S. Dollar Continue to Rise? Octa Br...,The U.S. dollar has been appreciating almost r...,2025-01-22,https://finance.yahoo.com/news/u-dollar-contin...
3,3,USD/JPY Poised for Another Peak?,Market Outlook: Global Indices.,2025-01-15,https://finance.yahoo.com/news/usd-jpy-poised-...
4,4,Here's 'the flip side' of the US dollar's stre...,The Federal Reserve cut interest rates by 25 b...,2024-12-19,https://finance.yahoo.com/video/heres-flip-sid...
5,5,Sterling Faces Challenges After Mostly Weaker ...,Preliminary GDP for the third quarter of 0.1% ...,2024-11-22,https://finance.yahoo.com/news/sterling-faces-...
6,6,F/X trades while central banks adjust policy: ...,The Japanese yen (JPY=X) weakened against the ...,2024-09-20,https://finance.yahoo.com/video/f-x-trades-whi...
7,7,USD/JPY Forecast – US Dollar Continues to Atte...,The US dollar has been sitting on the massive ...,2024-09-17,https://finance.yahoo.com/news/usd-jpy-forecas...
8,8,USD/JPY Forecast – US Dollar Continues to Test...,The US dollar has fallen a bit in the past 24 ...,2024-09-16,https://finance.yahoo.com/news/usd-jpy-forecas...
9,9,USD/JPY Weekly Price Forecast – US Dollar Cont...,The US dollar has plunged during the trading w...,2024-09-13,https://finance.yahoo.com/news/usd-jpy-weekly-...


In [ ]:
EURUSD = "EURUSD=X"
eur_usd_df = get_news_data(EURUSD)
today = date.today()
eur_usd_df.to_csv(f"{EURUSD}_{today}.csv")

In [ ]:
JPYUSD = "JPY=X"
jpy_usd_df = get_news_data(JPYUSD)
today = date.today()
jpy_usd_df.to_csv(f"{JPYUSD}_{today}.csv")

<time class="byline-attr-meta-time" data-timestamp="2025-02-06T15:07:59.000Z" datetime="2025-02-06T15:07:59.000Z">Thu, Feb 6, 2025, 8:07 AM</time>
<time class="byline-attr-meta-time" data-timestamp="2025-01-22T10:24:12.000Z" datetime="2025-01-22T10:24:12.000Z">Wed, Jan 22, 2025, 3:24 AM</time>
<time class="byline-attr-meta-time" data-timestamp="2025-01-15T15:12:14.000Z" datetime="2025-01-15T15:12:14.000Z">Wed, Jan 15, 2025, 8:12 AM</time>
<time class="byline-attr-meta-time" datetime="2024-12-19T15:33:19.000Z">Thu, Dec 19, 2024, 8:33 AM</time>
<time class="byline-attr-meta-time" data-timestamp="2024-11-22T10:08:26.000Z" datetime="2024-11-22T10:08:26.000Z">Fri, Nov 22, 2024, 3:08 AM</time>
<time class="byline-attr-meta-time" datetime="2024-09-20T16:14:25.000Z">Fri, Sep 20, 2024, 10:14 AM</time>
<time class="byline-attr-meta-time" data-timestamp="2024-09-17T13:12:18.000Z" datetime="2024-09-17T13:12:18.000Z">Tue, Sep 17, 2024, 7:12 AM</time>
<time class="byline-attr-meta-time" data-timesta

In [ ]:
GBPUSD = "GBPUSD=X"
gbp_usd_df = get_news_data(GBPUSD)
today = date.today()
gbp_usd_df.to_csv(f"{GBPUSD}_{today}.csv")

<time class="byline-attr-meta-time" data-timestamp="2025-03-09T06:00:28.000Z" datetime="2025-03-09T06:00:28.000Z">Sat, Mar 8, 2025, 11:00 PM</time>


## FXStreet.com

### Single Link Scraper

In [4]:
# get html data from a website
r = requests.get('https://www.fxstreet.com/news/eur-usd-price-analysis-bulls-push-higher-but-overbought-signals-hint-at-correction-202503071624')

In [5]:
soup = BeautifulSoup(r.text, 'html.parser')

In [6]:
soup

 <!DOCTYPE html>
 <html lang="en"> <head> <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><script type="text/javascript">window.NREUM||(NREUM={});NREUM.info = {"beacon":"bam.nr-data.net","errorBeacon":"bam.nr-data.net","licenseKey":"c2da35057c","applicationID":"492039093","transactionName":"YQZSbUNSD0cABRBeV1hMcWphHBJSDBAHR1lRBkNcQ0UIVwRJVVUMAFIFDQYeWQRZAEkBD1IBHQEFVQMZBwBUAggGBwkJAgdUaw8JEF9dWwZvXF8dAEcRHg==","queueTime":0,"applicationTime":393,"agent":"","atts":""}</script><script type="text/javascript">(window.NREUM||(NREUM={})).init={ajax:{deny_list:["bam.nr-data.net"]}};(window.NREUM||(NREUM={})).loader_config={licenseKey:"c2da35057c",applicationID:"492039093"};;/*! For license information please see nr-loader-rum-1.284.1.min.js.LICENSE.txt */
(()=>{var e,t,r={122:(e,t,r)=>{"use strict";r.d(t,{a:()=>i});var n=r(944);function i(e,t){try{if(!e||"object"!=typeof e)return(0,n.R)(3);if(!t||"object"!=typeof t)return(0,n.R)(4);const r=Object.create(Object.getPrototype

In [23]:
from dateutil import parser

date = soup.find("time").text
date # We'll store it the way it is, the raw form of the date object

'03/07/2025 16:24:00 GMT'

In [7]:
# Extract the article only 
articles = soup.find_all('article', class_=("fxs_article") )
articles

[<article class="fxs_article">
 <header class="fxs_article_header" id="fxs_article_header">
 <h1 class="fxs_headline_large">EUR/USD Price Analysis: Bulls push higher but overbought signals hint at correction</h1>
 <span class="fxs_entry_metaInfo">
 <a class="fxs_entry_sectionlink" data-gtmid="lateralnavigation-post-section" href="https://www.fxstreet.com/news">NEWS</a> | <time datetime="2025-03-07T16:24:00.0000000Z">03/07/2025 16:24:00 GMT</time>
                             |
                            By <a data-gtmid="lateralnavigation-post-author" href="/author/patricio-martin">Patricio Martín</a>
 </span>
 </header>
 </article>,
 <article class="fxs_article">
 <div class="fxs_article_body" id="fxs_article_body">
 <div class="fxs_article_content" id="fxs_article_content">
 <ul>
 <li><strong>EUR/USD holds gains after the European session, trading near the 1.0870 area and marking fresh highs since November 2024.</strong></li>
 <li><strong>The 20 and 100-day moving averages are neari

In [8]:
import re

paragraph_tags = r"<p>(.*?)</p>"
anchor_tags = r"<a>(.*?)</a>"

paragraphs_extracted = []
for article in articles: 
    paragraphs = re.findall(paragraph_tags, str(article), re.DOTALL)
    for paragraph in paragraphs:
        if paragraphs is not None: 
            paragraphs_extracted.append(paragraph)
            
paragraphs_extracted


['The EUR/USD pair stands higher on Friday after the European session, reaching its strongest level since November 2024 above 1.0800 as bullish momentum continued to build. Buyers remain firmly in control, driving the pair into fresh territory, with price action now reflecting strong upward momentum.',
 'The Relative Strength Index (RSI) is positioned in overbought territory and rising sharply, indicating strong buying pressure but also signaling that the rally could face exhaustion soon. Meanwhile, the Moving Average Convergence Divergence (MACD) is printing rising green bars, reinforcing the current bullish <a data-fxs-autoanchor="" href="https://www.fxstreet.com/rates-charts/forecast">outlook</a>. A notable development is the approaching bullish crossover between the 20-day and 100-day Simple Moving Averages (SMAs), which could further support buyers if confirmed.',
 'On the technical front, resistance is now seen near the 1.0900 zone, with a break above potentially opening the door

I ended up using regular expression because it was easier to manipulate the raw data in this format, using built-in native Python datatypes rather than manipulating via BeautifulSoup's objects.

In [9]:
# Extract irrelevant data, including disclaimer data and anchor data. 

anchor_tags = r"<a.*?</a>"
HTML_tags = r"<.*?>"

for idx, paragraph in enumerate(paragraphs_extracted):

    # Remove anchors
    anchor = re.search(anchor_tags, paragraph)
    if anchor:
        paragraph = paragraph.strip(anchor.string)
        paragraphs_extracted[idx] = paragraph
    
    # Remove newline characters
    newlines = re.findall(r"\n", paragraph)
    for newline in newlines:
        paragraph = paragraph.strip(newline)
        paragraphs_extracted[idx] = paragraph

    # Remove any leftover HTML tags
    tags = re.findall(HTML_tags, paragraph)
    for tag in tags:
        print("Tag found")
        paragraph = paragraph.strip(tag)
        paragraphs_extracted[idx] = paragraph

Tag found


In [10]:
paragraphs_extracted

['The EUR/USD pair stands higher on Friday after the European session, reaching its strongest level since November 2024 above 1.0800 as bullish momentum continued to build. Buyers remain firmly in control, driving the pair into fresh territory, with price action now reflecting strong upward momentum.',
 '',
 'On the technical front, resistance is now seen near the 1.0900 zone, with a break above potentially opening the door for further gains toward 1.0950. On the downside, immediate support stands around 1.0800, followed by 1.0700 and well below\xa0the 20 and 100-day SMA convergence near 1.0500. If selling pressure emerges, a pullback toward these levels could signal a technical correction before another bullish attempt.',
 '',
 'Information on these pages contains forward-looking statements that involve risks and uncertainties. Markets and instruments profiled on this page are for informational purposes only and should not in any way come across as a recommendation to buy or sell in t

Amalgamate

In [11]:
HTML_content = ""

for i in paragraphs_extracted:
    HTML_content += i

In [12]:
HTML_content # This is the text, we also want to push meta data in our data

'The EUR/USD pair stands higher on Friday after the European session, reaching its strongest level since November 2024 above 1.0800 as bullish momentum continued to build. Buyers remain firmly in control, driving the pair into fresh territory, with price action now reflecting strong upward momentum.On the technical front, resistance is now seen near the 1.0900 zone, with a break above potentially opening the door for further gains toward 1.0950. On the downside, immediate support stands around 1.0800, followed by 1.0700 and well below\xa0the 20 and 100-day SMA convergence near 1.0500. If selling pressure emerges, a pullback toward these levels could signal a technical correction before another bullish attempt.Information on these pages contains forward-looking statements that involve risks and uncertainties. Markets and instruments profiled on this page are for informational purposes only and should not in any way come across as a recommendation to buy or sell in these assets. You shou

### Per Page, Scrape the Link

In [29]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from icecream import ic

# Initialize WebDriver
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Run in headless mode (remove if debugging)
driver = webdriver.Chrome(options=options)

page = 0  # Goes up to 59
ticker = "EURUSD"

try:
    # Open FXStreet News page
    driver.get(f'https://www.fxstreet.com/news/latest/asset?q=&hPP=17&idx=FxsIndexPro&p={page}&dFR[Category][0]=News&dFR[Tags][0]={ticker}')

    # Wait for the page to fully load
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.TAG_NAME, 'body'))
    )

    # Handle pop-up if it appears
    try:
        close_button = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, "//*[contains(text(), 'CONTINUE TO SITE')]"))
        )
        close_button.click()
        print("Pop-up closed.")
    except:
        print("No pop-up found.")

    # Wait for news section to load
    WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, 'fxs_headline_tiny'))
    )

    # Scroll down to load more articles (if needed)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Extract news headlines and URLs
    headlines = driver.find_elements(By.CLASS_NAME, 'fxs_headline_tiny')

    news_data = []
    for headline in headlines:
        try:
            link = headline.find_element(By.TAG_NAME, "a")
            title = link.text.strip()
            url = link.get_attribute("href")
            news_data.append((title, url))
        except:
            print("Skipping an element due to missing link.")

    print("Articles were scraped successfully")

finally:
    driver.quit()  # Close the browser

ic(news_data)


Pop-up closed.
Skipping an element due to missing link.
Skipping an element due to missing link.
Skipping an element due to missing link.
Skipping an element due to missing link.
Skipping an element due to missing link.
Skipping an element due to missing link.
Skipping an element due to missing link.
Skipping an element due to missing link.
Articles were scraped successfully


ic| news_data: [('EUR/USD eases as Trump turns tariff threats to Europe',
                 'https://www.fxstreet.com/news/eur-usd-eases-as-trump-turns-tariff-threats-to-europe-202503132345'),
                ('Forex Today: Tariffs, German inflation, and UK GDP gather attention',
                 'https://www.fxstreet.com/news/forex-today-tariffs-german-inflation-and-uk-gdp-gather-attention-202503131842'),
                ('EUR/USD Price Analysis: Buyers take a breather as consolidation phase '
                 'begins',
                 'https://www.fxstreet.com/news/eur-usd-price-analysis-buyers-take-a-breather-as-consolidation-phase-begins-202503131543'),
                ('EUR/USD slumps on fears of potential US-EU tariff war',
                 'https://www.fxstreet.com/news/eur-usd-corrects-slightly-amid-fears-of-potential-eu-us-trade-war-202503131005'),
                ('EUR drifts as spreads widen slightly – Scotiabank',
                 'https://www.fxstreet.com/news/eur-drifts-a

[('EUR/USD eases as Trump turns tariff threats to Europe',
  'https://www.fxstreet.com/news/eur-usd-eases-as-trump-turns-tariff-threats-to-europe-202503132345'),
 ('Forex Today: Tariffs, German inflation, and UK GDP gather attention',
  'https://www.fxstreet.com/news/forex-today-tariffs-german-inflation-and-uk-gdp-gather-attention-202503131842'),
 ('EUR/USD Price Analysis: Buyers take a breather as consolidation phase begins',
  'https://www.fxstreet.com/news/eur-usd-price-analysis-buyers-take-a-breather-as-consolidation-phase-begins-202503131543'),
 ('EUR/USD slumps on fears of potential US-EU tariff war',
  'https://www.fxstreet.com/news/eur-usd-corrects-slightly-amid-fears-of-potential-eu-us-trade-war-202503131005'),
 ('EUR drifts as spreads widen slightly – Scotiabank',
  'https://www.fxstreet.com/news/eur-drifts-as-spreads-widen-slightly-scotiabank-202503131339'),
 ('EUR/USD: A leg higher may occur if Russia agrees to the truce terms – Danske Bank',
  'https://www.fxstreet.com/new

### Iterate Through All Available Pages

Last Page Found for EUR/USD: `https://www.fxstreet.com/news?q=&hPP=17&idx=FxsIndexPro&p=58&dFR[Tags][0]=EURUSD`

Where: `p=58` dictates the page number

Or we can do; if no articles are found within a page, then terminate the loop!

In [14]:
import pandas as pd

df = pd.read_parquet("EURUSD=X_fxstreet_database")
df

,title,link,date,text
0,EUR/USD Price Analysis: Euro extends losses bu...,https://www.fxstreet.com/news/eur-usd-price-an...,03/21/2025 15:50:00 GMT,The Relative Strength Index (RSI) has sharply ...
1,"EUR/USD slumps on Eurozone economic risks, Fed...",https://www.fxstreet.com/news/eur-usd-drops-as...,03/21/2025 14:39:04 GMT,Powell commented that the implementation of ne...
2,EUR drifts near range base – Scotiabank,https://www.fxstreet.com/news/eur-drifts-near-...,03/21/2025 13:23:43 GMT,Spot continues to consolidate in the low 1.08s...
3,EUR/USD may find good support at its 200-DMA –...,https://www.fxstreet.com/news/eur-usd-may-find...,03/21/2025 11:36:45 GMT,"EUR/USD remains heavy near this week’s lows, B..."
4,EUR/USD to consolidate in the 1.08-1.09 range ...,https://www.fxstreet.com/news/eur-usd-to-conso...,03/21/2025 10:28:54 GMT,"EUR/USD fell below 1.09 yesterday, Danske Bank..."
...,...,...,...,...
9,EUR: Waiting for ECB Monetary Policy Meeting A...,https://www.fxstreet.com/news/eur-waiting-for-...,10/10/2024 08:49:00 GMT,“Hence the market is now fully pricing 25bp ra...
10,EUR/USD: Expected to edge lower – UOB Group,https://www.fxstreet.com/news/eur-usd-expected...,10/10/2024 08:24:54 GMT,24-HOUR VIEW: “We expected EUR to trade in a s...
11,German Retail Sales rise 2.1% YoY in August,https://www.fxstreet.com/news/german-retail-sa...,10/10/2024 06:05:00 GMT,Germany's Retail Sales increased by 1.6% MoM i...
12,"EUR/USD hangs near two-month low, seems vulner...",https://www.fxstreet.com/news/eur-usd-hangs-ne...,10/10/2024 01:00:48 GMT,The EUR/USD pair oscillates in a narrow band b...


Changing `.parquet` saved databases into `datalake` 

In [4]:
from deltalake import DeltaTable, write_deltalake
import pandas as pd

df = pd.read_parquet("../data/raw_scrapped_data/EURUSD=X_fxstreet_database")
write_deltalake("../data/delta/EURUSD=X_fxstreet_database", df)

Now compare the file sizes

In [6]:
ticker = "EURUSD=X"

file_path = r'../data/raw_scrapped_data'
file_name = f"/{ticker}_fxstreet_database"

database = pd.read_parquet(file_path + file_name)

In [7]:
database

,title,link,date,text
0,EUR/USD Price Analysis: Euro extends losses bu...,https://www.fxstreet.com/news/eur-usd-price-an...,03/21/2025 15:50:00 GMT,The Relative Strength Index (RSI) has sharply ...
1,"EUR/USD slumps on Eurozone economic risks, Fed...",https://www.fxstreet.com/news/eur-usd-drops-as...,03/21/2025 14:39:04 GMT,Powell commented that the implementation of ne...
2,EUR drifts near range base – Scotiabank,https://www.fxstreet.com/news/eur-drifts-near-...,03/21/2025 13:23:43 GMT,Spot continues to consolidate in the low 1.08s...
3,EUR/USD may find good support at its 200-DMA –...,https://www.fxstreet.com/news/eur-usd-may-find...,03/21/2025 11:36:45 GMT,"EUR/USD remains heavy near this week’s lows, B..."
4,EUR/USD to consolidate in the 1.08-1.09 range ...,https://www.fxstreet.com/news/eur-usd-to-conso...,03/21/2025 10:28:54 GMT,"EUR/USD fell below 1.09 yesterday, Danske Bank..."
...,...,...,...,...
9,EUR: Waiting for ECB Monetary Policy Meeting A...,https://www.fxstreet.com/news/eur-waiting-for-...,10/10/2024 08:49:00 GMT,“Hence the market is now fully pricing 25bp ra...
10,EUR/USD: Expected to edge lower – UOB Group,https://www.fxstreet.com/news/eur-usd-expected...,10/10/2024 08:24:54 GMT,24-HOUR VIEW: “We expected EUR to trade in a s...
11,German Retail Sales rise 2.1% YoY in August,https://www.fxstreet.com/news/german-retail-sa...,10/10/2024 06:05:00 GMT,Germany's Retail Sales increased by 1.6% MoM i...
12,"EUR/USD hangs near two-month low, seems vulner...",https://www.fxstreet.com/news/eur-usd-hangs-ne...,10/10/2024 01:00:48 GMT,The EUR/USD pair oscillates in a narrow band b...


In [21]:
database1 = pd.read_parquet(r"../data/raw_scrapped_data/GBPUSD=X_fxstreet_database")
database1

,title,link,date,text
0,"GBP/USD slips as trader war fears, central ban...",https://www.fxstreet.com/news/gbp-usd-slips-as...,03/21/2025 15:35:00 GMT,"Kenneth Broux, the head of corporate research ..."
1,Pound Sterling underperforms US Dollar as Fed ...,https://www.fxstreet.com/news/pound-sterling-c...,03/21/2025 14:05:53 GMT,Fed Chair Jerome Powell said in the press conf...
2,GBP edges slightly lower – Scotiabank,https://www.fxstreet.com/news/gbp-edges-slight...,03/21/2025 13:28:58 GMT,Government borrowing data revealed a larger th...
3,GBP: BOE keeps the policy rate steady at 4.50%...,https://www.fxstreet.com/news/gbp-boe-keeps-th...,03/21/2025 11:33:14 GMT,"GBP/USD is down to a 5-day low, and 10-year gi..."
4,GBP: No major impact from BoE hold – ING,https://www.fxstreet.com/news/gbp-no-major-imp...,03/21/2025 11:16:15 GMT,"""We still look with some concern at next week’..."
...,...,...,...,...
15,GBP/USD: GBP rebounds from PMI-driven dip – Sc...,https://www.fxstreet.com/news/gbp-usd-gbp-rebo...,09/23/2024 12:55:00 GMT,UK PMI data reflected a softening in activity ...
16,GBP: Long GBP positioning is quite extreme – ING,https://www.fxstreet.com/news/gbp-long-gbp-pos...,09/23/2024 10:39:58 GMT,"The Bank of England's broad, trade-weighted st..."
0,GBP/USD: To trade between 1.3270 and 1.3340 – ...,https://www.fxstreet.com/news/gbp-usd-to-trade...,09/23/2024 09:52:23 GMT,The Pound Sterling (GBP) is likely to trade in...
1,UK Preliminary Services PMI dips to 52.8 in Se...,https://www.fxstreet.com/news/uk-preliminary-s...,09/23/2024 08:31:00 GMT,The seasonally adjusted S&amp;P Global/CIPS UK...


In [20]:
pd.read_parquet(r"C:\Users\sumag\Downloads\GBPUSD=X_fxstreet_database")

,title,link,date,text
0,GBP/USD rises above 1.2900 as US yields declin...,https://www.fxstreet.com/news/gbp-usd-rises-ab...,03/27/2025 02:32:00 GMT,"However, the upside of the GBP/USD pair could ..."
1,GBP/USD backslides as market sentiment sours,https://www.fxstreet.com/news/gbp-usd-backslid...,03/26/2025 22:55:20 GMT,"GBP/USD sewered on Wednesday, tumbling six-ten..."
2,"GBP/USD slides below 1.29 on soft UK CPI, budg...",https://www.fxstreet.com/news/gbp-usd-slides-b...,03/26/2025 15:30:00 GMT,The UK’s inflation rate dipped below estimates...
3,Pound Sterling slumps after UK budget update,https://www.fxstreet.com/news/pound-sterling-d...,03/26/2025 13:51:00 GMT,The Pound Sterling (GBP) faces selling pressur...
4,UK's Reeves: Real disposable income to grow th...,https://www.fxstreet.com/news/uks-reeves-real-...,03/26/2025 13:16:37 GMT,Information on these pages contains forward-lo...
...,...,...,...,...
15,GBP/USD: GBP rebounds from PMI-driven dip – Sc...,https://www.fxstreet.com/news/gbp-usd-gbp-rebo...,09/23/2024 12:55:00 GMT,UK PMI data reflected a softening in activity ...
16,GBP: Long GBP positioning is quite extreme – ING,https://www.fxstreet.com/news/gbp-long-gbp-pos...,09/23/2024 10:39:58 GMT,"The Bank of England's broad, trade-weighted st..."
0,GBP/USD: To trade between 1.3270 and 1.3340 – ...,https://www.fxstreet.com/news/gbp-usd-to-trade...,09/23/2024 09:52:23 GMT,The Pound Sterling (GBP) is likely to trade in...
1,UK Preliminary Services PMI dips to 52.8 in Se...,https://www.fxstreet.com/news/uk-preliminary-s...,09/23/2024 08:31:00 GMT,The seasonally adjusted S&amp;P Global/CIPS UK...
